In [ ]:
import numpy as np 
import pandas as pd   
### preprocessing data 
from sklearn.preprocessing import LabelEncoder 
#### spliting data
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
## mdoels
from xgboost import XGBClassifier 
from sklearn.ensemble import VotingClassifier 
from sklearn.ensemble import RandomForestClassifier 
#####
from sklearn.metrics import roc_auc_score 

## load data

In [ ]:
train_data = pd.read_csv("../input/spaceship-titanic/train.csv").drop(['PassengerId' , 'Name'] , axis=1) 
test_data =pd.read_csv("../input/spaceship-titanic/test.csv").drop(['PassengerId','Name'] , axis=1) 
sample =pd.read_csv('../input/spaceship-titanic/sample_submission.csv') 


# show data

In [ ]:
train_data.head(15)
  

In [ ]:
train_data.info()

In [ ]:
test_data.head()  

In [ ]:
test_data.describe().T

# data preprocessing 

first we take null values

In [ ]:
train_data.isna().sum().sort_values(ascending=False)

In [ ]:
test_data.isna().sum().sort_values(ascending=False)

# discrite data to object data and numerical data

 first we will deal with object features :

In [ ]:
obj_train_features=[]
for col in train_data.columns:
    if(np.dtype(train_data[col]) == 'object'):
        obj_train_features.append(col) 
print(len(obj_train_features))
        

In [ ]:
obj_test_features=[]
for col in test_data.columns:
    if(np.dtype(test_data[col]) == 'object'):
        obj_test_features.append(col) 
print(len(obj_test_features))

In [ ]:
num_train_features=[]
for col in train_data.columns:
    if(np.dtype(train_data[col]) == 'float64'):
        num_train_features.append(col) 
print(len(num_train_features))
        

In [ ]:
list(obj_train_features)

In [ ]:
list(obj_test_features)

In [ ]:
train_data['HomePlanet'].value_counts()    ### Europa , Earth ,Mars

In [ ]:
test_data['HomePlanet'].value_counts()  ### Europa , Earth ,Mars

in Homeplanet we full all nulls values with common value  (Earth)

In [ ]:
train_data['Destination'].value_counts()    ## TRAPPIST-1e , PSO J318.5-22 , 55 Cancri e

In [ ]:
test_data['Destination'].value_counts()    ## TRAPPIST-1e , PSO J318.5-22 , 55 Cancri e

in Destination full nulls values with TRAPPIST-1e value

In [ ]:
train_data["VIP"].value_counts()      ### common value is False

In [ ]:
train_data['CryoSleep'].value_counts()    ### common value is false 

map function switch value by value   so we used it to make columns is numerical 

make function to min time & memory 

In [ ]:
def clean(data):
    data['HomePlanet']= data['HomePlanet'].fillna('Earth')             ##  common value is (Earth)
    data['HomePlanet']= data['HomePlanet'].map({'Europa':0 , 'Earth':1 , 'Mars':2})
    data['Destination']=data['Destination'].fillna("TRAPPIST-1e")     ## common value is TRAPPIST-1e 
    data['Destination']=data['Destination'].map({'TRAPPIST-1e':0 , 'PSO J318.5-22':1 ,  '55 Cancri e':2})
    data['VIP']=        data["VIP"].fillna(False).astype(int)         ## common value is False
    data['CryoSleep']=  data['CryoSleep'].fillna(False).astype(int)   ## common value is False
    return data
    
train_data= clean(train_data)
test_data = clean(test_data)

then we deal with numerical data 

In [ ]:
list(num_train_features)

full all numerical features with mean of it

In [ ]:
def full_nuls(data):
    for col in num_train_features :
        data[col]=data[col].fillna(data[col].mean()) 
    return data 
train_data=full_nuls(train_data)
test_data = full_nuls(test_data)
train_data.isna().sum().sort_values(ascending =False)

after processing we just have one column not processing (Cabin) 
so we split this columns into two columns ( Deck_cabin , Sid_cabin )
1. Deck_cabin is split cabine by it is first part 
2. Sid_cabin  is split cabin by it is last part 


this function taken [from](https://www.kaggle.com/code/srikantsahoo/spaceship-titanic-eda-baseline-models?scriptVersionId=97078229&cellId=25) and modified it 

In [ ]:
def D_cabin(Cabin):
    try:
        return Cabin.split('/')[0]
    except:
        return np.NaN 
def S_cabin(Cabin):
    try :
        return Cabin.split('/')[2]
    except :
        return np.NaN
train_data['Dec_cabin'] = train_data["Cabin"].apply(lambda x: D_cabin(x))
train_data['Sid_cabin'] = train_data["Cabin"].apply(lambda x: S_cabin(x))
train_data = train_data.drop(['Cabin'] , axis=1) 

test_data['Dec_cabin'] = test_data["Cabin"].apply(lambda x: D_cabin(x))
test_data['Sid_cabin'] = test_data["Cabin"].apply(lambda x: S_cabin(x))
test_data = test_data.drop(['Cabin'] , axis=1) 



In [ ]:
train_data['Dec_cabin'].value_counts(dropna=False)

In [ ]:
#  F and G is bigest numbers so we full nulls values with it
train_data['Dec_cabin'].iloc[:2500] = train_data['Dec_cabin'].iloc[:2500].fillna("f", inplace =True)


train_data['Dec_cabin'].iloc[2500:] = train_data['Dec_cabin'].iloc[2500:].fillna("G", inplace =True)


In [ ]:
train_data['Sid_cabin'].value_counts( dropna=False)

In [ ]:
#  precentage is 50% to 50%  of S and P 
np.random.seed(43) 
data = np.random.choice(a = list(train_data["Sid_cabin"].value_counts().index) ,p  = [0.5,0.5] , size =199)


fill = pd.DataFrame(index= train_data.index[train_data['Sid_cabin'].isnull()] , data=data , columns =["Sid_cabin"]) 
train_data.fillna(fill ,inplace=True)

In [ ]:
test_data['Sid_cabin'].value_counts( dropna=False)

In [ ]:
#  precentage is 50% to 50%  of S and P 
np.random.seed(43) 
data = np.random.choice(a = list(test_data["Sid_cabin"].value_counts().index) ,p  = [0.5,0.5] , size =100)


fill = pd.DataFrame(index= test_data.index[test_data['Sid_cabin'].isnull()] , data=data , columns =["Sid_cabin"]) 
test_data.fillna(fill ,inplace=True)

In [ ]:
train_data['Sid_cabin'] = train_data['Sid_cabin'].map({'S':1 , 'P':0})
test_data['Sid_cabin'] = test_data['Sid_cabin'].map({'S':1 , 'P':0})

# encod label

In [ ]:
label_model = LabelEncoder() 
train_data['Dec_cabin'] = label_model.fit_transform(train_data['Dec_cabin'])

test_data['Dec_cabin'] = label_model.fit_transform(test_data['Dec_cabin'])

In [ ]:
# show data
train_data.head()

drop Transported from train data to split data 

In [ ]:
y=train_data['Transported']
train_data = train_data.drop(['Transported'] ,axis=1)
X  =train_data
X_test =test_data

# mdoel

In [ ]:
# preparing algorithms to use it
SKF_model = StratifiedKFold(n_splits=5)
XGB_model = XGBClassifier(n_estimators=100,learning_rate=0.07,booster='gbtree',gamma=0.5 ,
                          reg_alpha=0.5 , reg_lambda=0.5,base_score=0.2) 
RF_model = RandomForestClassifier(n_estimators=120 , criterion='gini' ,
                                  min_samples_split=1.0 ,min_samples_leaf=0.5 , max_leaf_nodes=4) 
VC_model = VotingClassifier(estimators=[('XGB' , XGB_model) , ("Rf" , RF_model)] ,  voting='hard') 

In [ ]:
test_list=[]
for count , (train_idx ,test_idx) in enumerate(SKF_model.split(X,y)):
    X_train = X.iloc[train_idx] 
    X_valid = X.iloc[test_idx] 
    y_train = y.iloc[train_idx]
    y_valid = y.iloc[test_idx] 
    print("*************fold(" , count+1 , ")***************")
    VC_model.fit(X_train , y_train)
    y_predict = VC_model.predict(X_valid) 
    test_predict = VC_model.predict(X_test) 
    score =roc_auc_score(y_valid , y_predict)
    test_list.append(test_predict)
    print('score------>' , score)
    print("\n")


so fold number four is best one so we choose it

In [ ]:
# show test predict
test_predict=test_list[3]
test_predict[:20]

# submission 

In [ ]:
sample['Transported']=test_predict 
sample.to_csv("submission.csv",index=False)
sample.head()